In [1]:
####################################
## A. Librerias
####################################
import os
import time
import json

import datetime
import pandas as pd

from utils.utils import *
from utils.paths import CHROMEDRIVER_PATH, PROJECT_PATH

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

####################################
## B. Funciones
####################################

def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def start_har(proxy, name):
    options = {
        'captureHeaders': True, 
        'captureContent': True, 
        'captureBinaryContent': True, 
        'captureCookies': True, 
        'captureHeadersSize': -1, 
        'captureMaxSize': -1, 
        'captureBinaryContentMaxLength': -1 # Tamaño máximo del contenido binario
    }
    proxy.new_har(name, options=options)

ESTAMOS EN --->  darwin
PROJECT_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper
CHROME_DRIVER_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver



In [2]:
########################
## C. Ejecución
########################

# 1. Seleccionar modo
HEADLESS = False
PRINT_VIEW = False
# CHROMEDRIVER_PATH = None

# 2. Eliminar todos los archivos de la carpeta
try:
    files = get_files(os.path.join(os.getcwd(),'outputs','jsons','renacyt'), '.')
    for file in files:
        os.remove(file)
    print('Se eliminaron los archivos de la carpeta')
except:
    pass

Se eliminaron los archivos de la carpeta
Ya existe  ./jsons
Se creó la carpeta ./jsons/renacyt_profile


In [6]:
######################
## D. Leer los jsons
######################

paths = get_files(os.path.join(os.getcwd(),'outputs','jsons','renacyt'), '.json')
df_renacyt = pd.concat([pd.read_json(path) for path in paths], axis=0)
df_renacyt

,id,codigoRegistro,tipoDocumento,numeroDocumento,apellidoPaterno,apellidoMaterno,nombres,emailNotificar,orcid,ctiVitae,...,convocatoriaId,fechaAnulacion,fechaInicioVigenciaAnt,fechaFinVigenciaAnt,grupoAnt,nivelAnt,fechaExclusion,estado,flagOrcid,fechaActualizacionOrcid
0,16281,P0005093,DNI,06203497,OSORIO,CARRERA,CESAR JAVIER,cosorio@concytec.gob.pe,0000-0002-2850-6420,5093,...,6,NaN,NaN,NaN,,,NaN,NaN,False,NaN
1,16280,P0223284,DNI,72840086,LEON,FIGUEROA,DARWIN ANTHONY,dalefi19@gmail.com,0000-0001-7267-0204,223284,...,6,NaN,1.685744e+12,1.748902e+12,N,III,NaN,NaN,False,NaN
2,16279,P0171899,DNI,73779093,CCAMI,BERNAL,FABRICIO JULIAN,fccami@unsa.edu.pe,0000-0003-3172-2113,171899,...,6,NaN,1.685116e+12,1.748274e+12,N,V,NaN,NaN,False,NaN
3,16278,P0125717,DNI,09789519,VALENZUELA,FERNANDEZ,LUIS ALEX,luisa.valenzuela@upsjb.edu.pe,0000-0002-8743-4092,125717,...,6,NaN,1.600924e+12,1.695532e+12,MR,II,NaN,NaN,False,NaN
4,16277,P0123317,DNI,00797710,DIAZ,ZELADA,YVAN FRANCISCO,yvandiazz@gmail.com,0000-0001-9149-068X,123317,...,6,NaN,1.608527e+12,1.703135e+12,MR,II,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7147,7580,P0177054,DNI,44386274,GONZALEZ,REAÑO,JORGE LUIS,jgonzalez@utec.edu.pe,0000-0003-0828-8347,177054,...,6,NaN,1.628129e+12,1.722823e+12,MR,II,NaN,NaN,False,NaN
7148,7579,P0160474,CARNET EXTRANJERIA,002219801,PARADA,QUINAYÁ,DIANA CAROLINA,dparada@utec.edu.pe,0000-0002-9928-5127,160474,...,6,NaN,1.628039e+12,1.691111e+12,CM,IV,NaN,NaN,False,NaN
7149,7578,P0160212,PASAPORTE,042976376,MARÍN,VELÁSQUEZ,TOMÁS DARÍO,tmarin@protonmail.com,0000-0002-3334-5895,160212,...,6,NaN,1.620398e+12,1.715093e+12,MR,II,NaN,NaN,False,NaN
7150,7574,P0117349,DNI,44489410,RUBIO,NORIEGA,RUTH ESTHER,ruthyrn@gmail.com,0000-0003-0869-8995,117349,...,6,NaN,1.593666e+12,1.656738e+12,CM,IV,NaN,NaN,False,NaN


In [3]:
######################
## D. Ejecución
######################

# 0. Definir los datos de la cuenta
url = 'https://servicio-renacyt.concytec.gob.pe/busqueda-de-investigadores/'

# 1. Iniciar el driver, el servidor proxy y el proxy
driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=PRINT_VIEW, headless=HEADLESS)

# 2. Iniciar har
start_har(proxy, 'renacyt')

# 3. Abrir la página
driver.get(url)

Se inicia el servidor en Mac o Linux
Usando el chromedriver_path: /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver


In [4]:
# 4. Obtener el .har y guardar los json
time.sleep(TIME_SLEEP*2)
har = proxy.har

In [5]:
i = 0
for entrie in har.get('log').get('entries'):
    aux_entrie = str(entrie)

    if '"codigoRegistro"' in aux_entrie and '"institucionLaboralPrincipal"' in aux_entrie:
        string_content = entrie.get('response').get('content').get('text')
        data_list = json.loads(string_content)
        with open(os.path.join(os.getcwd(),'jsons','renacyt','renacyt_' + str(i).zfill(2) + '_data.json'), 'w') as json_file:
            json.dump(data_list, json_file, indent=4)
        i += 1

In [6]:
stop_chrome_driver(driver, server, proxy)

In [7]:
# Eliminar los archivos bmp.log y server.log
try:
    os.remove('bmp.log')
    os.remove('server.log')
    print("Se eliminaron los archivos bmp.log y server.log")
except:
    pass

Se eliminaron los archivos bmp.log y server.log
